In [1]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

/opt/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def sigma(x):
    return 1./(1. + np.exp(-x))

In [4]:
class RBM:
    def __init__(self, input_size=784, hidden_size=100):
        xavier_range = np.sqrt(6.0/(input_size + hidden_size))
#         self.W = np.random.uniform(-xavier_range, xavier_range, (input_size, hidden_size))
        self.W = np.random.rand(input_size, hidden_size) * 0.5
        self.b = np.zeros((1,hidden_size))
        self.a = np.zeros((1,input_size))
        self.input_size = input_size
        self.hidden_size = hidden_size
    
    def front_propagation(self, input_visible):
        activated_hidden = sigma(np.matmul(input_visible, self.W) + self.b)
        rand_array = np.random.rand(activated_hidden.shape[0], activated_hidden.shape[1])
        h = (activated_hidden > rand_array).astype('int')
        return h
    
    def back_propagation(self, input_hidden):
        return sigma(np.matmul(input_hidden, np.transpose(self.W)) + self.a)
        
    
    def train(self, X, learning_rate=0.001, generating=False):
        number_of_data = X.shape[0]
        h = self.front_propagation(X)
        
        if generating:
            random_seed = np.random.randn(int(number_of_data/4), self.input_size)
            temp_1 = self.back_propagation(self.front_propagation(random_seed))
            temp_2 = self.back_propagation(self.front_propagation(temp_1))
            temp_3 = self.back_propagation(self.front_propagation(temp_2))
            temp_4 = self.back_propagation(self.front_propagation(temp_3))
            self.X1 = np.concatenate((temp_1, temp_2, temp_3, temp_4), axis=0)
        
        h1 = self.front_propagation(self.X1)
        
    
        dW = np.matmul(np.transpose(X), h) - np.matmul(np.transpose(self.X1), h1)
        db = np.sum(h - h1, axis=0, keepdims=True)
        da = np.sum(X - self.X1, axis=0, keepdims=True)
        
        self.W += learning_rate * dW / number_of_data
        self.b += learning_rate * db / number_of_data
        self.a += learning_rate * da / number_of_data

        error = np.mean(np.sum((X - self.X1) ** 2, axis=1))
        self.X1 = self.back_propagation(h1)
        
        return error
        
        
    def generate_images(self, number_of_images=20, n_step=200):
        hidden = np.round(np.random.rand(number_of_images, self.hidden_size))
        for i in range(n_step):
            visible = self.back_propagation(hidden)
            hidden = self.front_propagation(visible)
        return visible
    
    def generate_hidden(self, X, n_step=200):
        visible = X
        for i in range(n_step):
            hidden = self.front_propagation(visible)
            visible = self.back_propagation(hidden)
        return hidden

In [5]:
model = RBM(hidden_size =30)

In [6]:
BATCH_SIZE = 100
TRAINING_EPOCHS = 50

In [7]:
for epoch in range(TRAINING_EPOCHS):
    total_batch = int(mnist.train.num_examples / BATCH_SIZE)
    total_error = 0
    check = True
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
        total_error += model.train(batch_xs, generating=check)/total_batch
        if check:
            check = False
    print('Epoch: ' + str((epoch + 1)) + ': ' + str(total_error) )

print('Learning Finished!')

Epoch: 1: 350.311711524
Epoch: 2: 114.429178331
Epoch: 3: 103.178315692
Epoch: 4: 91.0829579889
Epoch: 5: 82.1992003559
Epoch: 6: 75.2840551598
Epoch: 7: 73.7057658782
Epoch: 8: 70.8393772321
Epoch: 9: 69.0463640413
Epoch: 10: 64.8479114732
Epoch: 11: 63.4013485728
Epoch: 12: 62.58848694
Epoch: 13: 59.9430317071
Epoch: 14: 59.2990587202
Epoch: 15: 61.0670205292
Epoch: 16: 58.6353263604
Epoch: 17: 60.1990269507
Epoch: 18: 62.042835466
Epoch: 19: 60.246441129
Epoch: 20: 59.1280412389
Epoch: 21: 60.6153687627
Epoch: 22: 62.0403583696
Epoch: 23: 60.4214981031
Epoch: 24: 61.2923951547
Epoch: 25: 59.8684510043
Epoch: 26: 59.9167908286
Epoch: 27: 60.0890815993
Epoch: 28: 61.094012106
Epoch: 29: 61.4278625283
Epoch: 30: 61.0541124829
Epoch: 31: 63.164065776
Epoch: 32: 61.8794658365
Epoch: 33: 61.7297856133
Epoch: 34: 63.3786313684
Epoch: 35: 62.5184443914
Epoch: 36: 62.8944869636
Epoch: 37: 63.0521557516
Epoch: 38: 62.8462312744
Epoch: 39: 62.871540866
Epoch: 40: 63.4837805973
Epoch: 41: 63.78

In [ ]:
reshaped_images = np.reshape(model.generate_images(number_of_images=100, n_step=1000), (-1, 28, 28))
# for image in reshaped_images:
#     plt.imshow(image, cmap='gray')
#     plt.show()
f, axarr = plt.subplots(10, 10, figsize=(10,10))
for i in range(10):
    for j in range(10):
        axarr[i, j].imshow(reshaped_images[i*10 + j])
        axarr[i, j].axis('off')
        axarr[i, j].set_xticklabels([])
        axarr[i, j].set_yticklabels([])
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [8]:
hidden_array = []
for i in range(total_batch):
    images = mnist.train.next_batch(BATCH_SIZE)[0]
    hidden = sigma(np.matmul(images, model.W) +  model.b)
    hidden = (hidden > 0.5).astype('int')
    hidden_array.append(hidden)
hidden_array = np.concatenate(hidden_array).tolist()
hidden_string_array = []
for data in hidden_array:
    string = ''
    for row in data:
        string += str(row)
    hidden_string_array.append(string)
from collections import Counter
counter = Counter(hidden_string_array)
p_x = np.array(list(counter.values()))/len(hidden_array)
entropy_h = np.sum(-p_x * np.log(p_x))

counter_2 = Counter(counter.values())
p_x2 = np.array([k*v/len(hidden_array) for k, v in counter_2.items()])
entropy_k = np.sum(-p_x2 * np.log(p_x2))
entropy_h, entropy_k

(8.1187668099932022, 4.1216199674912994)